<h2 style="color:red">Imports</h2>

In [71]:
#imports 
#from tensorflow import keras
import keras
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import LSTM,Dense,Input,Bidirectional
from nltk.tokenize.treebank import TreebankWordTokenizer
from scipy import spatial
from random import shuffle
import pickle
from keras.callbacks import ModelCheckpoint
from scipy import spatial
import csv
import requests
import json
from textblob import TextBlob

<h2 style="color:red">Word Embedding</h2>

In [5]:
embeddings_index = {}
embeddings_size = 300
i = 0
with open('/home/adel/Downloads/chatBot/cc.fa.300.vec') as f:
    for line in f:
        if i > 400000:
            break
        i += 1
        values = line.split()
        word = values[0]
        coefs = [float(i) for i in values[1:]]
        embeddings_index[word] = coefs
embeddings_index['<PAD>'] = [0] * 300
embeddings_index['<UNK>'] = [1] * 300

In [15]:
def sim(dataSetI , dataSetII):
    return 1 - spatial.distance.cosine(dataSetI, dataSetII)

<h2 style="color:red">Preparing our Data</h2>

In [118]:
marks = {'؟', '!', '.', '،'}
zamir_1 = {'م' , 'ش', 'ت', 'ه','و','ی'} 
zamir_3 = {'تان', 'شان', 'مان'}
def prepareSent(sent):
    sent = sent.split()
    sent = [i[:-1] if i[-1] in marks or (i[-1] in zamir_1 and i[:-1] in embeddings_index and i not in embeddings_index) else i for i in sent]
    sent = [i[:-3] if i[-3:] in zamir_3 and i[:-3] in embeddings_index and i not in embeddings_index else i for i in sent]
    tokenized = []
    n = 0
    while(n != len(sent)):
        if n < len(sent) - 2 and (sent[n] + "‌" + sent[n+1] + "‌" + sent[n+2]) in embeddings_index:
            tokenized.append(sent[n] + "‌" + sent[n+1] + "‌" + sent[n+2])
            n += 2
        elif n != len(sent) -1 and (sent[n] + "‌" + sent[n+1]) in embeddings_index:
            tokenized.append(sent[n] + "‌" + sent[n+1])
            n += 1
        else:
            tokenized.append(sent[n])
        n += 1
    return tokenized

In [21]:
class_size = 5
X_train = []
Y_train = []
for i in range(1,class_size+1):
    with open('class{}.csv'.format(i)) as f:
        reader = csv.reader(f)
        class_list = [r[0] for r in reader]
        y = [1 if j == i else 0 for j in range(1 , class_size+1)]
        class_list = [(o,y) for o in class_list]
        X_train = X_train + class_list

In [22]:
shuffle(X_train)
shuffle(X_train)
shuffle(X_train)

In [23]:
Y_train = [i[1] for i in X_train]
X_train = [prepareSent(i[0]) for i in X_train]

In [24]:
MAX_SEQ = 20
def wordToVec(data):
    for s in range(len(data)):
        n = MAX_SEQ - len(data[s])
        if n < 0:
            data[s] = data[s][:MAX_SEQ]
        else:
            for i in range(n):
                data[s].append('<PAD>')
        for v in range(len(data[s])):
            if data[s][v] not in embeddings_index:
                data[s][v] = embeddings_index['<UNK>']
            else:
                data[s][v] = embeddings_index[data[s][v]]
    return np.array(data)

In [25]:
X_train = wordToVec(X_train)
X_train.shape

(503, 20, 300)

<h2 style="color:red">Building model with Batch size 64</h2>


In [26]:
BATCH_SIZE = 16
input_layer = Input( batch_shape = (None, MAX_SEQ, 300))
lstm_layer = Bidirectional(LSTM(units=MAX_SEQ))(input_layer)
output_layer = Dense(class_size, activation="softmax")(lstm_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='categorical_crossentropy',
              optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 20, 300)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 40)                51360     
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 205       
Total params: 51,565
Trainable params: 51,565
Non-trainable params: 0
_________________________________________________________________


In [27]:
X_train = X_train[:500]
Y_train = np.array(Y_train[:500])

In [28]:
checkpoint = ModelCheckpoint('weight_farsi_dekhte.{epoch:02d}.hdf5', period=25)
callbacks_list = [checkpoint]

In [29]:
#fit the model
EPOCH_SIZE = 50
model.fit(X_train, Y_train, epochs=EPOCH_SIZE, batch_size=BATCH_SIZE, callbacks=callbacks_list)

Epoch 1/50
500/500 [==============================] - 3s 7ms/step - loss: 1.5697
Epoch 2/50
500/500 [==============================] - 1s 2ms/step - loss: 1.4266
Epoch 3/50
500/500 [==============================] - 1s 2ms/step - loss: 1.1189
Epoch 4/50
500/500 [==============================] - 1s 2ms/step - loss: 0.8214
Epoch 5/50
500/500 [==============================] - 1s 2ms/step - loss: 0.5814
Epoch 6/50
500/500 [==============================] - 1s 2ms/step - loss: 0.4248
Epoch 7/50
500/500 [==============================] - 1s 2ms/step - loss: 0.3061A: 1s 
Epoch 8/50
500/500 [==============================] - 1s 2ms/step - loss: 0.2167
Epoch 9/50
500/500 [==============================] - 1s 2ms/step - loss: 0.1562
Epoch 10/50
500/500 [==============================] - 1s 2ms/step - loss: 0.1164
Epoch 11/50
500/500 [==============================] - 1s 2ms/step - loss: 0.1089
Epoch 12/50
500/500 [==============================] - 1s 2ms/step - loss: 0.0881
Epoch 13/50
500/500

In [ ]:
#defining entities
embeddings_size = 300
entity_lists = { "cloth" : ['شلوار' , 'کاپشن' , 'پیراهن'],
                "city_iran" : ['شیراز', 'کرج', 'مشهد'],
                "food" : ['کباب', 'ساندویچ', 'سوپ'],
                "time" : ['امروز', 'فردا', 'دیروز']
               }
for ent in entity_lists:
    sum_of_embedding = np.zeros(embeddings_size)
    for obj in entity_lists[ent]:
        sum_of_embedding += np.array(embeddings_index[obj])
    sum_of_embedding /= len(entity_lists[ent])
    globals()['entity_{}'.format(ent)] = list(sum_of_embedding)

In [ ]:
#recognize entity
def return_entity(sent , entity):
    sent = sent.lower()
    ma = 0
    ans = ""
    for i in sent.split():
        if sim(embeddings_index[i] , entity) > ma:
            ma = sim(embeddings_index[i] , entity)
            ans = i
    if ma < .3:
        return "nothing"
    return ans

In [30]:
def address(sent):
    sent = sent.replace(' ','+')
    res = requests.get("https://www.google.com/maps/search/?api=1&query={}+اصفهان&hl=fa".format(sent))
    print(res.url)

In [164]:
def weather(sent):
    city = return_entity(sent, entity_city_iran)
    if city == 'nothing':
        city = 'اصفهان'
    blob = TextBlob(city)
    blob = blob.translate(to="en")
    res = requests.get("https://api.openweathermap.org/data/2.5/weather?q={}&appid=3bb1d3931ea6593a0833bd5cf0b97ac3".format(blob))
    if res.status_code == 200:
        data = json.loads(res.text)
        temp = int(data['main']['temp'] - 273.15)
        desc = data['weather'][0]['description']
        blob = TextBlob(desc)
        desc = blob.translate(to="fa")
        print('هوای  {} {}'.format(city, desc))
        print('و دمای هوا {} درجه سانتیگراد می‌باشد'.format(temp))

In [31]:
def whichClass(inp):
    for i in range(len(inp)):
        if 1 == inp[i]:
            if i == 0:
                return "Address"
            elif i == 1:
                return "Resturant"
            elif i == 2:
                return "Home Service"
            elif i == 3:
                return "Laundary"
            elif i == 4:
                return "Weather"

In [141]:
def classify(sent):
    sentence = prepareSent(sent)
    sent = ' '.join(list(sentence))
    sentence = wordToVec([sentence])[0]
    sentence = np.reshape(sentence , (1 , 20 , 300))
    
    sentence = model.predict(sentence)
    argmax = np.argmax(sentence)
    a = [1 if argmax == i else 0 for i in range(class_size)]
    res = whichClass(list(a))
    if res == 'Address':
        address(sent)
    elif res == 'Weather':
        weather(sent)
    else:
        return res

In [39]:
sent = "میدونی چجوری باید برم سی و سه پل"
sent = 'رستوران خوب تو اصفهان هست؟'
classify(sent)

https://www.google.com/maps/search/?api=1&query=%D8%B1%D8%B3%D8%AA%D9%88%D8%B1%D8%A7%D9%86+%D8%AE%D9%88%D8%A8+%D8%AA%D9%88+%D8%A7%D8%B5%D9%81%D9%87%D8%A7%D9%86+%D9%87%D8%B3%D8%AA%D8%9F+%D8%A7%D8%B5%D9%81%D9%87%D8%A7%D9%86&hl=fa


In [163]:
sent = 'باران دوست داری ؟'
# prepareSent('میاد؟')
classify(sent)

اصفهان
هوای  اصفهان چند ابرها
و دمای هوا 35 درجه سانتیگراد می‌باشد
